In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import pyspark.sql.functions as F
import time

In [3]:
import os
import time
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
warnings.filterwarnings(action='ignore')
spark = SparkSession.builder.getOrCreate()

##Add "eagerEval.enabled" to beautify the way Spark DF is displayed
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

## To use legacy casting notation for date
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [5]:
bucket_read = 'msca-bdp-tweets'
folder_name = 'final_project'

In [6]:
cmd = 'hadoop fs -du -s -h ' + 'gs://' + bucket_read + '/' + folder_name + '/'

In [7]:
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

498.7 G  498.7 G  gs://msca-bdp-tweets/final_project



In [8]:
!hadoop fs -ls 'gs://msca-bdp-tweets/final_project/' | head

Found 50696 items
-rwx------   3 root root          0 2023-02-08 13:58 gs://msca-bdp-tweets/final_project/_SUCCESS
-rwx------   3 root root    4500466 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00000-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4107431 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00001-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4672123 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00002-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    5186684 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00003-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4729662 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00004-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4605529 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00005-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx-----

In [9]:
%time raw_df = spark.read.json('gs://' + bucket_read + '/' + folder_name)

23/03/07 21:56:48 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 1.32 s, sys: 294 ms, total: 1.61 s
Wall time: 7min 27s


23/03/07 22:02:20 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
#raw_df.select(count('*').alias('Total data')).show()

In [10]:
%%time
filtered_raw = raw_df\
    .filter('user.location is not null')\
    .filter('user.followers_count > 0')\
    .filter('user.favourites_count > 0')\
    .filter('withheld_in_countries is NULL')\
    .filter('truncated == "False"')\
    .filter('lang == "en"')\
    .withColumn("tweet_text", F.lower(F.col("tweet_text")))\
    .withColumn('stripped', F.regexp_replace(F.col("tweet_text"),"[\;\@\$\#\,&%\"\.\']|(\b'\w)|(\w'\b)|https\S+|http\S+|www\.\S+|\\\\\S*",""))

print("Count ",filtered_raw.count())

Count  46698507
CPU times: user 640 ms, sys: 135 ms, total: 775 ms
Wall time: 3min 27s


In [11]:
filtered_words = ["elementary education", "kindergarten", "preschool", "middle school", "junior high school", 
                  "high school", "secondary education", "senior high school", "higher education", "college", 
                  "university", "community college", "vocational school", "school district", "k-12","study abroad",
                  "education system", "curriculum", "lesson plan", "classroom","teacher", "professor", "instructor", 
                  "educator", "teaching assistant", "student", "principal", "grader","board of education", 
                  "entrance exam", "gpa", "report card", "transcript", "primary school", "degree", "diploma",     
                  "certificate", "major", "minor", "academic program", "undergraduate", "graduate", "doctorate", 
                  "thesis", "library", "textbook", "hybrid learning","exchange program", "bilingual education", 
                  "stem education", "engineering", "mathematics","international education"]

In [12]:
filtered_words_df='|'.join(["(" + c +")" for c in filtered_words])

In [13]:
non_edu = [ "robbery", "burglary", "theft", "fraud", "corruption", "scandal", "money laundering","shoot","kill","killed","gambling", "poker", "deceased","murder","attack","horny", 
           "aviation", "trucking", "mass transit","shooting","shootings","gunned","gun","guns","uvalde","urban planning",
           "distributors", "wholesalers", "retailers"]

In [14]:
non_edu_df='|'.join(["(" + c +")" for c in non_edu])

In [15]:
new_filtered = filtered_raw.where(filtered_raw['tweet_text'].rlike(filtered_words_df)).\
where(~filtered_raw['tweet_text'].rlike(non_edu_df))

In [16]:
new_filtered.count()

24074764

In [ ]:
%time new_filtered.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/sirivanth/new_filtered')

CPU times: user 2.46 s, sys: 574 ms, total: 3.03 s
Wall time: 16min 42s
